# Project 1

An Agent-Based SEIR Model

- Robert Morain
- Vigynesh Bhatt

### Abstract

Summarize what you did and what you learned

### Introduction

Short summary of why the problem is interesting to you (if it’s not interesting to
you, make something up), what you did, and how you organized your report.

### Experiment Conditions

I specified most of the experiment conditions, but I asked you to do
two things: first, choose parameters for your scale-free networks and justify those choices, and
second, run some experiments where you choose the infectiousness levels, time spent infectious,
and time spent exposed.

### Hypotheses 

Make some hypotheses about how the virus will propagate across the different
networks under the different conditions. Give some justification for your hypotheses in terms of
network characteristics (e.g., the metrics in your summary table).

### Discussion

Summarize which hypotheses were supported by data and which were not supported.
Explain why you think the results came out the way they did. If you are speculating about why,
state that you are hypothesizing a possible explanation. If the reason why is justified by the
data, tell me how the data supports your explanation.

### Future Work

Tell me what you wish you had done or could do now that the project is over.

In [14]:
import numpy as np
import networkx as nx

In [15]:
def p_infectious(d, p_1c, B):
    a = p_1c / (1 - p_1c)
    b = a * np.exp(B * d ** 3 - 1)
    return (a * b) / (1 + a * b) 

In [13]:
m_e = 1.0
s_e = 1.0
m_i = 2.25
s_i = 0.105
B = -0.0050367
p_1c = 0.12

In [21]:
class Agent():
    # TODO
    def __init__(self, countdown_to_infectious, countdown_to_recovered, days_spent_infectious, initial_state="S"):
        self.countdown_to_infectious = countdown_to_infectious
        self.countdown_to_recovered = countdown_to_recovered
        self.days_spent_infectious = days_spent_infectious
        self.initial_state = initial_state
        self.possible_states = ["S", "E", "I", "R"]
        self.current_state = initial_state
        
    def step(self):
        pass
        

In [22]:
class Population():
    # TODO
    def __init__(self):
        pass
    
    def step_all(self):
        pass

In [25]:
def _read_graph_from_file(filename="ia-infect-dublin/ia-infect-dublin.mtx"):
    fo = open(filename,"r")
    line = fo.readline() # Read file header
    line = fo.readline() # Number of vertices and edges
    if not line:
        print("error -- illegal format for input")
        return
    v = line.split(" ")
    numVertices = int(v[0])
    G = nx.Graph()
    G.add_nodes_from(range(1, numVertices+1))
    while True:
        line = fo.readline()
        if not line:
            break
        #print("Line{}: {}".format(count,line.strip()))
        v = line.split(" ")
        v1 = int(v[0])
        v2 = int(v[1])
        G.add_edge(v1,v2)
        G.add_edge(v2,v1)
    fo.close()
    return G